In [1]:
import MDAnalysis
import MDAnalysis.lib.NeighborSearch as NeighborSearch
import MDAnalysis.lib.mdamath as mdamath
import warnings
import matplotlib as mlp
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append("../baseuniverse/")
from BaseUniverse import BaseUniverse

In [4]:
class OrderParameterEnsemble(BaseUniverse):
    """A class used to perform analysis on Clusters of molecules

    Attributes
    ----------
    universe : MDAnalysis universe object
        Universe of the simulated system 
    cluster_objects : list of str
        Strings used for the definition of species which form 
        clusters. Can be atom names or molecule names.
    cluster_list : list of list of MDAnalysis ResidueGroups
        a list of ResidueGroups forms one cluster at a given time,
        for multiple times a list of these lists is produced.

    Methods
    -------
    cluster_analysis(cut_off=7.5, style="atom", measure="b2b", 
                     principal_axis="static")
        Calculates which molecules are clustering together for all
        timesteps. 
    """
    
    def __init__(self, coord, traj, cluster_objects):
        """
        Parameters
        ---------- 
        coord : string 
            Path to a coordinate-like file. E.g. a gromacs tpr or 
            gro file
        traj : string
            Path to a trajectory like file. E.g. a xtc or trr file. 
            Needs to fit the coord file
        cluster_objects : list of string
            Strings used for the definition of species which form 
            clusters. Can be atom names or molecule names.
        """
        super().__init__(coord, traj, cluster_objects)

    def calc_nematic_op(self, times=None, style="molecule", principal_axis="inertial"):
        """High level function clustering molecules together
        
        Example
        -------
        No Example yet

        Note
        ----
        Not all of the functionality is actually coded yet

        Parameters
        ----------
        cut_off : float, optional
            Minimal distance for two particles to be in the
            same cluster, in Angstroem. Results still depend 
            on the measure parameter.
        time : list of floats, optional
            If None, do for whole trajectory. If an interval
            is given like this (t_start, t_end) only do from start
            to end.
        style : string, optional
            "atom" or "molecule". Dependent on this, the 
            cluster_objects attribute is interpreted as molecule
            or atoms within a molecule. 
        principal_axis : string, optional
            "inertial" or "end to end". Defines the principal molecular axis as either the end to end vector of the molecule or the dominant axis of the inertial tensor.

        Raises
        ------ 
        NotImplementedError
            If an unspecified principal axis is choosen
        
        ToDo
        ----
        
        """
        self.universe = self._get_universe(self._coord, traj=self._traj)

        self.selected_species = self._select_species(self.universe,
                                                            style=style)
        
        self.cluster_list = []

        # Initialise the neighboursearch object
        self.neighbour_search = NeighborSearch.AtomNeighborSearch(
            self.selected_species, 
            box=self.universe.dimensions,
            bucket_size=10
            )

        if principal_axis == "inertial":
            self.principal_axis = self._get_inertial_axis
        elif principal_axis == "end to end":
            self.principal_axis = self._get_end_to_end
        else:
            raise NotImplementedError("{:s} is unspecified principal axis".format(principal_axis))
        
        # Loop over all trajectory times
        for time in self.universe.trajectory:
            if times is not None:
                if time.time > max(times) or time.time < min(times):
                    continue
            principal_axis_list = self.principal_axis(self.selected_species)
            print("****TIME: {:8.2f}".format(time.time))

        # Rewind Trajectory to beginning for other analysis
        self.universe.trajectory.rewind()

    def _get_inertial_axis(self, selected_species):
        """ Get list of principal intertial axis of selected_species
        
        """
        print(selected_species)
        print(selected_species.principal_axes(pbc=True))
        return 0

    def _calc_inertia_tensor():
        return 0

In [5]:
tpr  = "files/run.tpr"
traj = "files/traj.xtc"

OrderParamEns = OrderParameterEnsemble(tpr, traj, ["ME"])
OrderParamEns.calc_nematic_op()

<AtomGroup [<Atom 1: GLY of type GLY of resname ME, resid 0 and segid seg_0_ME>, <Atom 2: GLY of type GLY of resname ME, resid 0 and segid seg_0_ME>, <Atom 3: ES of type ES of resname ME, resid 0 and segid seg_0_ME>, ..., <Atom 17498: CM of type CM of resname ME, resid 1749 and segid seg_0_ME>, <Atom 17499: CM of type CM of resname ME, resid 1749 and segid seg_0_ME>, <Atom 17500: CE of type CE of resname ME, resid 1749 and segid seg_0_ME>]>
[[ 0.01525411 -0.03856088  0.99913982]
 [-0.21033922  0.97677217  0.04090892]
 [-0.97750945 -0.21078232  0.00678892]]
****TIME:     0.00
<AtomGroup [<Atom 1: GLY of type GLY of resname ME, resid 0 and segid seg_0_ME>, <Atom 2: GLY of type GLY of resname ME, resid 0 and segid seg_0_ME>, <Atom 3: ES of type ES of resname ME, resid 0 and segid seg_0_ME>, ..., <Atom 17498: CM of type CM of resname ME, resid 1749 and segid seg_0_ME>, <Atom 17499: CM of type CM of resname ME, resid 1749 and segid seg_0_ME>, <Atom 17500: CE of type CE of resname ME, resid 

<AtomGroup [<Atom 1: GLY of type GLY of resname ME, resid 0 and segid seg_0_ME>, <Atom 2: GLY of type GLY of resname ME, resid 0 and segid seg_0_ME>, <Atom 3: ES of type ES of resname ME, resid 0 and segid seg_0_ME>, ..., <Atom 17498: CM of type CM of resname ME, resid 1749 and segid seg_0_ME>, <Atom 17499: CM of type CM of resname ME, resid 1749 and segid seg_0_ME>, <Atom 17500: CE of type CE of resname ME, resid 1749 and segid seg_0_ME>]>
[[ 0.00497154  0.00131435  0.99998678]
 [-0.01328362 -0.99991082  0.0013803 ]
 [ 0.99989941 -0.01329031 -0.00495364]]
****TIME: 140000.00
<AtomGroup [<Atom 1: GLY of type GLY of resname ME, resid 0 and segid seg_0_ME>, <Atom 2: GLY of type GLY of resname ME, resid 0 and segid seg_0_ME>, <Atom 3: ES of type ES of resname ME, resid 0 and segid seg_0_ME>, ..., <Atom 17498: CM of type CM of resname ME, resid 1749 and segid seg_0_ME>, <Atom 17499: CM of type CM of resname ME, resid 1749 and segid seg_0_ME>, <Atom 17500: CE of type CE of resname ME, resid

<AtomGroup [<Atom 1: GLY of type GLY of resname ME, resid 0 and segid seg_0_ME>, <Atom 2: GLY of type GLY of resname ME, resid 0 and segid seg_0_ME>, <Atom 3: ES of type ES of resname ME, resid 0 and segid seg_0_ME>, ..., <Atom 17498: CM of type CM of resname ME, resid 1749 and segid seg_0_ME>, <Atom 17499: CM of type CM of resname ME, resid 1749 and segid seg_0_ME>, <Atom 17500: CE of type CE of resname ME, resid 1749 and segid seg_0_ME>]>
[[-0.00330137  0.01725957  0.99984559]
 [-0.04800229  0.99869569 -0.01739822]
 [-0.99884177 -0.04805232 -0.00246857]]
****TIME: 290000.00
<AtomGroup [<Atom 1: GLY of type GLY of resname ME, resid 0 and segid seg_0_ME>, <Atom 2: GLY of type GLY of resname ME, resid 0 and segid seg_0_ME>, <Atom 3: ES of type ES of resname ME, resid 0 and segid seg_0_ME>, ..., <Atom 17498: CM of type CM of resname ME, resid 1749 and segid seg_0_ME>, <Atom 17499: CM of type CM of resname ME, resid 1749 and segid seg_0_ME>, <Atom 17500: CE of type CE of resname ME, resid

<AtomGroup [<Atom 1: GLY of type GLY of resname ME, resid 0 and segid seg_0_ME>, <Atom 2: GLY of type GLY of resname ME, resid 0 and segid seg_0_ME>, <Atom 3: ES of type ES of resname ME, resid 0 and segid seg_0_ME>, ..., <Atom 17498: CM of type CM of resname ME, resid 1749 and segid seg_0_ME>, <Atom 17499: CM of type CM of resname ME, resid 1749 and segid seg_0_ME>, <Atom 17500: CE of type CE of resname ME, resid 1749 and segid seg_0_ME>]>
[[ 0.01166475  0.02346924  0.99965651]
 [ 0.24819375  0.96837127 -0.02563086]
 [-0.96864018  0.24840747  0.00547089]]
****TIME: 430000.00
<AtomGroup [<Atom 1: GLY of type GLY of resname ME, resid 0 and segid seg_0_ME>, <Atom 2: GLY of type GLY of resname ME, resid 0 and segid seg_0_ME>, <Atom 3: ES of type ES of resname ME, resid 0 and segid seg_0_ME>, ..., <Atom 17498: CM of type CM of resname ME, resid 1749 and segid seg_0_ME>, <Atom 17499: CM of type CM of resname ME, resid 1749 and segid seg_0_ME>, <Atom 17500: CE of type CE of resname ME, resid

<AtomGroup [<Atom 1: GLY of type GLY of resname ME, resid 0 and segid seg_0_ME>, <Atom 2: GLY of type GLY of resname ME, resid 0 and segid seg_0_ME>, <Atom 3: ES of type ES of resname ME, resid 0 and segid seg_0_ME>, ..., <Atom 17498: CM of type CM of resname ME, resid 1749 and segid seg_0_ME>, <Atom 17499: CM of type CM of resname ME, resid 1749 and segid seg_0_ME>, <Atom 17500: CE of type CE of resname ME, resid 1749 and segid seg_0_ME>]>
[[ 0.00456167  0.01143674  0.99992419]
 [-0.0617141   0.99803177 -0.01113355]
 [-0.99808344 -0.06165864  0.0052585 ]]
****TIME: 570000.00
<AtomGroup [<Atom 1: GLY of type GLY of resname ME, resid 0 and segid seg_0_ME>, <Atom 2: GLY of type GLY of resname ME, resid 0 and segid seg_0_ME>, <Atom 3: ES of type ES of resname ME, resid 0 and segid seg_0_ME>, ..., <Atom 17498: CM of type CM of resname ME, resid 1749 and segid seg_0_ME>, <Atom 17499: CM of type CM of resname ME, resid 1749 and segid seg_0_ME>, <Atom 17500: CE of type CE of resname ME, resid

<AtomGroup [<Atom 1: GLY of type GLY of resname ME, resid 0 and segid seg_0_ME>, <Atom 2: GLY of type GLY of resname ME, resid 0 and segid seg_0_ME>, <Atom 3: ES of type ES of resname ME, resid 0 and segid seg_0_ME>, ..., <Atom 17498: CM of type CM of resname ME, resid 1749 and segid seg_0_ME>, <Atom 17499: CM of type CM of resname ME, resid 1749 and segid seg_0_ME>, <Atom 17500: CE of type CE of resname ME, resid 1749 and segid seg_0_ME>]>
[[ 0.03832207 -0.02316518  0.99899689]
 [-0.02704131 -0.99938917 -0.02213696]
 [ 0.99889949 -0.02616585 -0.03892508]]
****TIME: 710000.00
<AtomGroup [<Atom 1: GLY of type GLY of resname ME, resid 0 and segid seg_0_ME>, <Atom 2: GLY of type GLY of resname ME, resid 0 and segid seg_0_ME>, <Atom 3: ES of type ES of resname ME, resid 0 and segid seg_0_ME>, ..., <Atom 17498: CM of type CM of resname ME, resid 1749 and segid seg_0_ME>, <Atom 17499: CM of type CM of resname ME, resid 1749 and segid seg_0_ME>, <Atom 17500: CE of type CE of resname ME, resid

<AtomGroup [<Atom 1: GLY of type GLY of resname ME, resid 0 and segid seg_0_ME>, <Atom 2: GLY of type GLY of resname ME, resid 0 and segid seg_0_ME>, <Atom 3: ES of type ES of resname ME, resid 0 and segid seg_0_ME>, ..., <Atom 17498: CM of type CM of resname ME, resid 1749 and segid seg_0_ME>, <Atom 17499: CM of type CM of resname ME, resid 1749 and segid seg_0_ME>, <Atom 17500: CE of type CE of resname ME, resid 1749 and segid seg_0_ME>]>
[[ 0.02731534 -0.01537268  0.99950866]
 [ 0.00451714  0.99987343  0.01525484]
 [-0.99961666  0.00409823  0.02738133]]
****TIME: 850000.00
<AtomGroup [<Atom 1: GLY of type GLY of resname ME, resid 0 and segid seg_0_ME>, <Atom 2: GLY of type GLY of resname ME, resid 0 and segid seg_0_ME>, <Atom 3: ES of type ES of resname ME, resid 0 and segid seg_0_ME>, ..., <Atom 17498: CM of type CM of resname ME, resid 1749 and segid seg_0_ME>, <Atom 17499: CM of type CM of resname ME, resid 1749 and segid seg_0_ME>, <Atom 17500: CE of type CE of resname ME, resid

<AtomGroup [<Atom 1: GLY of type GLY of resname ME, resid 0 and segid seg_0_ME>, <Atom 2: GLY of type GLY of resname ME, resid 0 and segid seg_0_ME>, <Atom 3: ES of type ES of resname ME, resid 0 and segid seg_0_ME>, ..., <Atom 17498: CM of type CM of resname ME, resid 1749 and segid seg_0_ME>, <Atom 17499: CM of type CM of resname ME, resid 1749 and segid seg_0_ME>, <Atom 17500: CE of type CE of resname ME, resid 1749 and segid seg_0_ME>]>
[[ 0.01453457 -0.02250033 -0.99964118]
 [ 0.08036547  0.99653867 -0.021262  ]
 [ 0.99665949 -0.0800276   0.01629251]]
****TIME: 1000000.00
